In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Load Datasets

In [2]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [3]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

X_multi, y_multi = load_data('data/tictac_multi.txt')

## KNN Classifier

In [4]:
def finetune_clf_parameters(X_train, y_train):
    model = KNeighborsClassifier()

    # Define the parameter grid
    param_grid = {'n_neighbors': np.arange(1, 10),
                  'weights': ['uniform', 'distance'],
                  'metric': ['minkowski', 'euclidean']}

    # RandomizedSearchCV to find the best parameters
    randomized_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=10, random_state=42)
    randomized_search.fit(X_train, y_train)

    # Get the best value of k
    best_params = randomized_search.best_params_
    print(best_params)
        
    return best_params

In [5]:
def knn_clf_train(X, y, is_one_tenth = False):
    # Split into training and testing data
    if is_one_tenth == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, shuffle=True, random_state=42)

    # Find the best parameters
    best_params = finetune_clf_parameters(X_train, y_train)

    # Define and train model
    model = KNeighborsClassifier(**best_params)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx

In [6]:
def print_clf_results(val_accuracy, test_accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of KNN Classification on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_accuracy)
    print("Test Accuracy = ", test_accuracy)
    print("Confusion Matrix:")
    print(confusion_mtrx.round(decimals=3)) # Round to 3 decimal places

## Final Dataset

In [7]:
val_acc_final, test_acc_final, confusion_mtrx_final = knn_clf_train(X_final, y_final)
print_clf_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

print("Extra Credit #2 - Train the models on 1/10th of the data")
val_acc_final, test_acc_final, confusion_mtrx_final = knn_clf_train(X_final, y_final, True)
print_clf_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

{'weights': 'distance', 'n_neighbors': 2, 'metric': 'euclidean'}
Performance of KNN Classification on Final Dataset:
Cross Validation Accuracy =  1.0
Test Accuracy =  1.0
Confusion Matrix:
[[1. 0.]
 [0. 1.]]
Extra Credit #2 - Train the models on 1/10th of the data
{'weights': 'distance', 'n_neighbors': 2, 'metric': 'euclidean'}
Performance of KNN Classification on Final Dataset:
Cross Validation Accuracy =  0.861111111111111
Test Accuracy =  0.8864426419466975
Confusion Matrix:
[[0.78  0.22 ]
 [0.055 0.945]]


## Single Dataset

In [8]:
val_acc_single, test_acc_single, confusion_mtrx_single = knn_clf_train(X_single, y_single)
print_clf_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

print("Extra Credit #2 - Train the models on 1/10th of the data")
val_acc_single, test_acc_single, confusion_mtrx_single = knn_clf_train(X_single, y_single, True)
print_clf_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

{'weights': 'distance', 'n_neighbors': 9, 'metric': 'euclidean'}
Performance of KNN Classification on Single Dataset:
Cross Validation Accuracy =  0.8715648854961833
Test Accuracy =  0.9061784897025171
Confusion Matrix:
[[0.95  0.003 0.006 0.    0.031 0.    0.006 0.    0.003]
 [0.006 0.881 0.018 0.018 0.03  0.    0.012 0.006 0.03 ]
 [0.043 0.005 0.909 0.021 0.005 0.    0.011 0.005 0.   ]
 [0.009 0.026 0.026 0.889 0.017 0.    0.    0.    0.034]
 [0.04  0.01  0.    0.    0.95  0.    0.    0.    0.   ]
 [0.026 0.079 0.013 0.026 0.    0.855 0.    0.    0.   ]
 [0.03  0.    0.04  0.01  0.01  0.    0.909 0.    0.   ]
 [0.    0.12  0.    0.04  0.02  0.04  0.02  0.76  0.   ]
 [0.09  0.034 0.011 0.011 0.    0.    0.    0.022 0.831]]
Extra Credit #2 - Train the models on 1/10th of the data
{'weights': 'distance', 'n_neighbors': 9, 'metric': 'euclidean'}
Performance of KNN Classification on Single Dataset:
Cross Validation Accuracy =  0.6014918414918414
Test Accuracy =  0.5980325644504749
Confusi

# KNN Regressor

In [9]:
def finetune_reg_parameters(X_train, y_train):
    knn_regressor = KNeighborsRegressor()
    multioutput_regressor = MultiOutputRegressor(knn_regressor)

    # Define the parameter grid
    param_grid = {'estimator__n_neighbors': np.arange(1, 10),
                  'estimator__weights': ['uniform', 'distance'],
                  'estimator__metric': ['minkowski', 'euclidean']}

    # RandomizedSearchCV to find the best value of k
    randomized_search = RandomizedSearchCV(multioutput_regressor, param_grid, n_iter=10, cv=10, scoring='neg_mean_squared_error')
    randomized_search.fit(X_train, y_train)

    # Get the best value of k
    best_params = randomized_search.best_params_
    print(best_params)
        
    return best_params

In [10]:
def knn_reg_train(X, y, is_one_tenth = False):
    # Split into training and testing data
    if is_one_tenth == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, shuffle=True, random_state=42)

    # Find the best parameters
    best_params = finetune_reg_parameters(X_train, y_train)

    # Remove estimator__ in front of every hyperparameter
    remove_str = "estimator__"
    
    for param in list(best_params.keys()):
        if remove_str in param:
            new_param = param.replace(remove_str, "")
            best_params[new_param] = best_params[param]
            del best_params[param]

    # Define and train model
    base_regressor = KNeighborsRegressor(**best_params)
    model = MultiOutputRegressor(base_regressor)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_rmse = cross_val_score(model, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
    val_rmse = np.mean(val_rmse * -1)  # Convert to positive

    # Get test accuracy
    test_accuracy = model.score(X_test, y_test)

    # Get RMSE
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    return val_rmse, test_accuracy, rmse

In [11]:
def print_reg_results(val_rmse, test_accuracy, rmse, dataset_name):
    print(f"Performance of KNN Regression on {dataset_name}:")
    print("Cross Validation RMSE = ", val_rmse)
    print("Test Accuracy = ", test_accuracy)
    print("RMSE = ", rmse)

## Multi Dataset

In [12]:
val_rmse_multi, test_acc_multi, rmse_multi = knn_reg_train(X_multi, y_multi)
print_reg_results(val_rmse_multi, test_acc_multi, rmse_multi, "Multi Dataset")

print("Extra Credit #2 - Train the models on 1/10th of the data")
val_rmse_multi, test_acc_multi, rmse_multi = knn_reg_train(X_multi, y_multi, True)
print_reg_results(val_rmse_multi, test_acc_multi, rmse_multi, "Multi Dataset")

{'estimator__weights': 'distance', 'estimator__n_neighbors': 9, 'estimator__metric': 'minkowski'}
Performance of KNN Regression on Multi Dataset:
Cross Validation RMSE =  0.03542710084410962
Test Accuracy =  0.829980034205453
RMSE =  0.16690767106061566
Extra Credit #2 - Train the models on 1/10th of the data
{'estimator__weights': 'distance', 'estimator__n_neighbors': 9, 'estimator__metric': 'euclidean'}
Performance of KNN Regression on Multi Dataset:
Cross Validation RMSE =  0.10097217976779074
Test Accuracy =  0.40517672391156345
RMSE =  0.314836435944949
